# Yolo-v2-tiny DW2TF validation

In [1]:
import numpy as np
import tensorflow as tf

# Random input data to test 
in_data = np.random.randn(64, 416, 416, 3) * 120.56

### Output from Darkflow converted Yolo-v2-tiny

In [2]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/darkflow/yolov2-tiny.ckpt.meta')
ckpt_path = '/data/darkflow/yolov2-tiny.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('input:0')
out_t = g.get_tensor_by_name('output:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)

    out_data_darkflow = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/darkflow/yolov2-tiny.ckpt


### Output from DW2TF converted Yolo-v2-tiny

In [3]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/DW2TF/data/yolov2-tiny.ckpt.meta')
ckpt_path = '/data/DW2TF/data/yolov2-tiny.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('yolov2-tiny/net1:0')
out_t = g.get_tensor_by_name('yolov2-tiny/convolutional9/BiasAdd:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)
    
    out_data_dw2tf = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/DW2TF/data/yolov2-tiny.ckpt


### RMS Error

In [4]:
print(out_data_darkflow.shape)
print(out_data_dw2tf.shape)
print(np.mean(out_data_darkflow - out_data_dw2tf)**2)

(64, 13, 13, 425)
(64, 13, 13, 425)
3.734651923107484e-14


In [5]:
print(out_data_darkflow[0,0,0,:10])

[-32.597694   13.941672    4.1781635   2.0687065 -85.162506  139.02223
  61.937466  125.50001    83.98407    36.48591  ]


In [6]:
print(out_data_dw2tf[0,0,0,:10])

[-32.59783    13.941475    4.178166    2.0686564 -85.161705  139.0219
  61.936897  125.500015   83.98348    36.486507 ]
